# Distance based time series classification in aeon

\emph{Distance based} classifiers use a distance function to measure the similarity
between time series. Aeon provides a range of time series specific distance measures
in the distances module (see LINK distances notebook).




## 1. Imports

In [1]:
from sktime.datasets import load_basic_motions, load_italy_power_demand

## 2. Load data

In [2]:
X_train, y_train = load_italy_power_demand(split="train", return_X_y=True)
X_test, y_test = load_italy_power_demand(split="test", return_X_y=True)
X_test = X_test[:50]
y_test = y_test[:50]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_mv, y_train_mv = load_basic_motions(split="train", return_X_y=True)
X_test_mv, y_test_mv = load_basic_motions(split="test", return_X_y=True)

X_train_mv = X_train_mv[:20]
y_train_mv = y_train_mv[:20]
X_test_mv = X_test_mv[:20]
y_test_mv = y_test_mv[:20]

print(X_train_mv.shape, y_train_mv.shape, X_test_mv.shape, y_test_mv.shape)

(67, 1) (67,) (50, 1) (50,)
(20, 6) (20,) (20, 6) (20,)
